**24007965, 24010268, 24005911, 24006362**

Before you start work on the project, **[click on this link to read the MATH0011 project instructions.](https://www.ucl.ac.uk/~ucahmto/0011/project-instructions-2025.html)**



# Fourier series and the Gibbs phenomenon

This project is about [Fourier series](https://en.wikipedia.org/wiki/Fourier_series), although you don't need any previous knowledge of Fourier series (covered in [MATH0008 Applied Mathematics 1](https://www.ucl.ac.uk/maths/sites/maths/files/math0008.pdf) and [MATH0016 Methods 3](https://www.ucl.ac.uk/maths/sites/maths/files/math0016.pdf)) to complete it. By doing this project you will implement the (extremely useful) mathematical techinque of computing Fourier series, gain experience in plotting using `matplotlib`, learn how to do numerical integration with `scipy`, and practise reading the documentation for Python modules.

### Recap on Fourier series

In your analysis modules you looked at power series representations for functions - for example, you can represent the function $f(x) = e^x$ by a series
$$ \sum_{i=0}^\infty \frac{x^i}{i!}.$$
A power series for a function $f$ expresses $f(x)$ as a sum of multiples of powers of $x$.  Fourier series are similar, but they represent functions as sums of multiples of functions of the form $\sin(nx)$ and $\cos(mx)$.

The coefficients in the Taylor series for a function $f$ are related to the derivatives of $f$ at 0: the Taylor series about $x=0$ for a smooth function $f$ is $\sum_{n=0}^\infty \frac{f^{n}(0)}{n!} x^n$.  The coefficients in the Fourier series for a function are obtained by integration.  Specifically, the Fourier series for a function $f$ on the interval $[0, 2\pi]$ is

$$ a_0 + \sum_{n=1}^\infty a_n \cos(nx) + \sum_{n=1}^\infty b_n \sin(nx)  \;\;\;\;\;\; (\dagger)$$

where the coefficients are given by $a_0 = \frac{1}{2\pi} \int_0^{2\pi} f(x) \;dx$, and for $n\geqslant 1$,

$$ a_n = \frac{1}{\pi} \int_0 ^{2\pi} f(x) \cos(nx)\; dx \;\;\;\;\; \text{and} \;\;\;\;\; b_n = \frac{1}{\pi} \int_0 ^{2\pi} f(x) \sin(nx)\; dx. $$

Just like a Taylor series, the Fourier series ($\dagger$) may or may not converge to $f(x)$ - what happens depends on $f$ and on $x$. In this project you will use numerical integration to compute Fourier coefficients, plot partial sums of Fourier series to see how the series behave as we add more and more terms, look at the Gibbs pheonomenon which describes how the partial sums of Fourier series behave at points of discontinuity of the function, and how to mitigate it.

The first cell below imports some modules you may need.

In [3]:
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
from scipy.integrate import quad

## Part 1 - numerical integration and Fourier coefficients

Read [the `scipy` documentation](https://docs.scipy.org/doc/scipy/index.html) to find out how to do numerical integration using `scipy.integrate`.  You don't need to do anything complex here - basic usage of the `scipy.integrate` methods will produce excellent results for the functions we will work with.  As a test, **write code in the next cell to numerically compute and print out approximations of the values of $\int_0^1 x^2 \; dx$, $\int_0^\infty e^{-x^2} \; dx$, and $\int_0^{2\pi} \sin(6x) \; dx$.**  Your answers should be approximately $1/3$, $\sqrt{\pi}/2$, and $0$.

In [4]:
def f(x):
    return x**2
#defining the function for the first integral required
def g(x):
    return np.exp(-(x**2))
#defining the function for the second integral required
def h(x): 
    return np.sin(6*x)
#defining the function for the third integral required
integral1 = quad(f, 0, 1)
integral2 = quad(g, 0, np.inf)
integral3 = quad(h, 0, (2*(np.pi)))

#calculatng integrals using the quad function 

print (integral1[0], integral2[0], integral3[0]) 
# used the [0] to get the first position to exclude the estimation in the output

0.33333333333333337 0.8862269254527579 1.9050237047800507e-15


**Using `scipy.integrate`, write functions `a0(f)`, `a(n, f)`, and `b(n, f)` which return the Fourier coefficients $a_0$, $a_n$, and $b_n$ of a function $f : \mathbb{R} \to \mathbb{R}$ as defined in the introduction to this notebook.**

Don't forget the constants $\frac1\pi$ and $\frac{1}{2\pi}$ outside the integrals!

It will help you to express yourself concisely in Python if you learn about `lambda`s.  You don't need to do this to answer this part of the project, but if you want to you can read about them [here](https://www.w3schools.com/python/python_lambda.asp).

In [5]:
def a0(f):
    result = quad (f, 0, (2*np.pi))
    return (1/(2*np.pi))*(result[0])
# using the local variable "result" to do the integration then using return to output the value of a0

In [6]:
def a(n, f):
    def integrand(x):                       
        return f(x) * np.cos(n*x)
    #we defined a function in a function so we can use it to multiply f(x) with the cos function
    result = quad (integrand, 0, (2*np.pi))
    #using the integrand function to integrate
    return (1/(np.pi))*(result[0])
    
  

In [7]:
def b(n, f):
    def integrand(x):
        return f(x) * np.sin(n*x)
    #we defined a function in a function so we can use it to multiply f(x) with the sin function
    result = quad (integrand, 0, (2*np.pi))
    #using the integrand function to integrate
    return (1/(np.pi))*(result[0])
    

Think of a way to test your functions `a0`, `a(n, f)`, and `b(n, f)` and do it in the cell below.

In [8]:
def f(x):
    return x**2
#test

print(a0(f))

13.159472534785811


## Part 2 - partial sums of Fourier series

"The $k$th partial sum of the Fourier series of a function $f$" means the function sending a number $x$ to

$$\texttt{fourier\_partial\_sum}(k, f, x) = a_0 + \sum_{n=1}^k a_n \cos(nx) + \sum_{n=1}^k b_n\sin(nx)$$

where $a_i$ and $b_i$ are the Fourier coefficients of $f$ defined as in the introduction to this project.

**Using your answers to part 1, write a function `fourier_partial_sum(k, f, x)` which computes the $k$th partial sum of the Fourier series of $f$ at $x$.**

In [9]:
def fourier_partial_sum(k, f, x):
    sum_a = 0
    sum_b = 0
   
    for n in range(1,k+1):
        sum_a +=(a(n, f))*np.cos(n*x)
        sum_b +=  b(n, f)*(np.sin(n*x))
        #using the for loop to do the summation
    return a0(f)+ sum_a + sum_b
    #returning the sum of summation values 

In [10]:
#Tests
def f(x):
    return x**2

fourier_partial_sum(2, f, (np.pi)/4)



0.8189484760356844

In [11]:
#Tests
sum_a = 0
k = 2
x = (np.pi)/4
for n in range(1,k):
        
    sum_a += sum_a + (a(n, f)*np.cos(n*x))
print (sum_a)

2.828427124746191


In [12]:
#Test
sum_b = 0
k = 2
x = (np.pi)/4
for n in range(1,k):
        
    sum_b += sum_b + (b(n, f)*np.sin(n*x))
print (sum_b)

-8.885765876316732


## Part 3 - computing and plotting Fourier partial sums

The square wave function is defined on the interval $[0, 2\pi]$ by
$$ \mathtt{square\_wave}(x) = \begin{cases} 
\pi/4 & 0 \leqslant x \leqslant \pi \\
-\pi/4 & \pi < x \leqslant 2\pi\end{cases}$$

**Write a Python function that computes the square wave function defined above.**  You don't need to consider inputs outside the interval $[0, 2\pi]$.

In [13]:
def square_wave(x):
    if x<=np.pi:
        out = math.pi/4
    else:
        out = -math.pi/4
    return out
# generating the correct values for the square wave

In [14]:
square_wave(np.pi) #test

0.7853981633974483

Now **plot the square wave function on $[0, 2\pi]$ and the $k$th partial sum of its Fourier series for $k=1, 5, 10$ on the same axes.**

Use your answer to part 2 to compute the partial sums.  Include a legend or some other method to indicate which line on your plot is which.  Don't use too many $x$-values: experiment to find the minimum number that makes the partial sums look smooth. If you use too many, the plot will take a long time.

In [15]:
xs = np.linspace(0, 2*np.pi, 100)
ys = [square_wave(x) for x in xs]
plt.plot(xs, ys,'black', label = "Square Wave")
# plotting the square wave function

rs = [fourier_partial_sum(1, square_wave, x) for x in xs]
qs = [fourier_partial_sum(5, square_wave, x) for x in xs]
zs = [fourier_partial_sum(10, square_wave, x) for x in xs]
# generating y values for the 1st, 5th and 10th Fourier partial sum 

plt.plot(xs,rs, 'r+', label = "1st Partial Sum")
plt.plot(xs,qs, 'b:', label = "5th Partial Sum")
plt.plot(xs,zs, 'g--', label = "10th Partial Sum")
plt.title("Square Wave Function Approximation")
# labelling functions and graph

plt.legend()
plt.grid()

The triangle wave function is defined on the interval $[0, 2\pi]$ by
$$ \mathtt{triangle\_wave}(x) = \begin{cases} 
\pi x/4 & 0 \leqslant x < \pi/2 \\
\pi^2/8 - \frac{\pi}{4}(x-\pi/2) & \pi/2 \leqslant x < 3\pi/2 \\
-\pi^2/8 + \frac{\pi}{4}(x-3\pi/2) & 3\pi/2 \leqslant x \leqslant 2\pi.
\end{cases}$$

**Write a Python function that computes the triangle wave function above.**  You don't need to consider inputs outside the interval $[0, 2\pi]$.

In [16]:
def triangle_wave(x):
    if 0<=x<((np.pi)/2):
        return (np.pi)*x/4
    elif np.pi/2<=x<3*((np.pi)/2):
        return (((np.pi)**2)/8)-((np.pi)/4)*(x-(np.pi)/2)
    elif 3*((np.pi)/2)<= x <= 2*np.pi:
        return -(((np.pi)**2)/8)+((np.pi)/4)*(x-(3*((np.pi))/2))
    # defining triangle wave function

In [17]:
triangle_wave(np.pi) #test

0.0

In [18]:
triangle_wave(3*(np.pi/2)) #test

-1.2337005501361697

**Plot the triangle wave function on $[0, 2\pi]$ and the $k$th partial sums of its Fourier series for $k = 1, 3, 5$ on the same axes.**

In [19]:
xs = np.linspace(0, 2*np.pi, 100)
ys = [triangle_wave(x) for x in xs]
plt.plot(xs,ys,'black', label = "Triangle Wave")
# plotting the Triangle wave function

ts = [fourier_partial_sum(1, triangle_wave, x) for x in xs]
ws = [fourier_partial_sum(3, triangle_wave, x) for x in xs]
zs = [fourier_partial_sum(5, triangle_wave, x) for x in xs]
# generating y values for the 1st, 3rd and 5th Triangle partial sum 

plt.plot(xs,ts,'r+', label = "1st Partial Sum")
plt.plot(xs,ws,'b--', label = "3rd Partial Sum")
plt.plot(xs,zs,'g:', label = "5th Partial Sum")
plt.title("Triangle Wave Function Approximation")
# labelling functions and graph

plt.legend()
plt.grid()
# importing the grid and key

## Part 4 - the Gibbs phenomenon

**Use the `scipy` numerical integration functions to compute an approximation to $I = \int_0 ^\pi \operatorname{sinc}(t) \mathrm{d}t$**, where

$$ \operatorname{sinc}(x) = \begin{cases} 1 & x = 0 \\ \frac{\sin(x)}{x} & x \neq 0. \end{cases}$$

In [20]:
def sinc(x):
    if x == 0:
        out = 1
    else:
        out = ((np.sin(x))/x)
    return out
# defining sinc(x) function

In [21]:
sinc(np.pi/2) #test

0.6366197723675814

In [22]:
I = integrate.quad(sinc,0,np.pi)[0] # integrating sinc(x) between pi and 0. used the [0] to get the first position to exclude the estimation in the output

**On the same axes, with $x$-values between $0$ and $2\pi$, plot the square wave, its 30th Fourier partial sum, and the two lines $y = \pm I/2$.**

Do not use more than a few hundred x-values in your plot - it may be too slow.

In [23]:
xs = np.linspace(0,2*np.pi,100)
ys = [I/2 for x in xs]
zs = [-I/2 for x in xs]
plt.plot(xs,ys, label = "+I/2")
plt.plot(xs,zs, label= "-I/2")
# plotting and labelling +I/2 and -I/2 

ws = [fourier_partial_sum(30, square_wave, x) for x in xs]
vs = [square_wave(x) for x in xs]
plt.plot(xs,ws, label = "30th Partial Sum")
plt.plot(xs,vs,  label = "Square Wave")
plt.title("The Gibbs Phenomenon")
# plotting and labelling the 30th Partial Sum, Square wave, and the title 

plt.legend()
plt.grid()
# importing the grid and key

You should see the [**Gibbs pheonomenon**](https://en.wikipedia.org/wiki/Gibbs_phenomenon) appear on your plot: the partial sum of the Fourier series is larger in magnitude than the square wave it is approximating near points where the square wave changes value.  This is a serious issue in signal processing where these undershoots and overshoots result in visible or audible *ringing* artifacts.  One way to reduce it is via the *Lanczos sigma factor*.

The $k$th Lanczos partial sum for a function $f: [0, 2\pi] \to \mathbb{R}$ is defined as follows:

$$\mathtt{lanczos\_partial\_sum}(x,k) = a_0 + \sum_{n=1}^k \operatorname{sinc}\left(\frac{n\pi}{2(k+1)}\right) a_n \cos(nx) + \sum_{n=1}^k \operatorname{sinc}\left(\frac{n\pi}{2(k+1)}\right) b_n \sin(nx)$$

where the $a_i$ and $b_i$ are the coefficients in the ordinary Fourier series.  The extra factor in each of the summands of the two sums is called the Lanczos sigma factor.  **Write a function `lanczos_partial_sum(k, f, x)` which computes the $k$th Lanczos partial sum of a function $f$.**


In [24]:
def lanczos_partial_sum(k, f, x):
    sum_c = 0
    for n in range(1,k+1):
        sum_c += (sinc((n*np.pi)/(2*(k+1))))*a(n,f)*(np.cos(n*x))
        
    sum_d = 0
    for n in range(1,k+1):
        sum_d += (sinc((n*np.pi)/(2*(k+1))))*b(n,f)*(np.sin(n*x))
    return a0(f) + sum_c + sum_d
   # defining Lanczos partial sum 
    

**On the same axes, plot `lanczos_partial_sum(x, square_wave, 30)`, the square wave function `square_wave(x)`, and the lines $y = \pm \frac I2$.**   Again, don't use more than a few hundred x-values.

You should see that the Gibbs phenomenon is reduced but not eliminated. 

In [25]:
xs = np.linspace(0, 2*np.pi, 100)
ys = [I/2 for x in xs]
zs = [-I/2 for x in xs]
plt.plot(xs,ys, label = "+I/2")
plt.plot(xs,zs, label = "-I/2")
# plotting and labelling +I/2 and -I/2 

ws = [lanczos_partial_sum(30, square_wave, x) for x in xs]
vs = [square_wave(x) for x in xs]
plt.plot(xs,ws, 'r-', label = "Lanczos 30th Partial Sum")
plt.plot(xs,vs, 'g', label = "Square Wave")
plt.title("Reduced Gibbs Phenomenon")
# plotting and labelling the Lanczos 30th Partial Sum, Square wave, and the title 

plt.grid()
plt.legend()
# importing the grid and key

# Submitting your project

Check you have done the following.

0. Included **all** group members' student numbers at the top of this notebook.
1. Read through every exercise to check you have answered every part.
2. Carefully read and followed all of the [MATH0011 project instructions](https://www.ucl.ac.uk/~ucahmto/0011/project-instructions-2025.html).
3. Checked that all of the code in this notebook works correctly.

One group member should download the completed notebook (in CoCalc, click the File menu next to the green Save button, then click Download) and submit it on the MATH0011 Moodle page in the projects section.  Please submit **only one file per group.**